# 🌍 Simple Disaster COG Processing

This simplified notebook converts disaster satellite imagery to Cloud Optimized GeoTIFFs (COGs) with just a few cells.

## ✨ Features
- **See files first** - List S3 files before configuring
- **Smart configuration** - Define filename functions after seeing actual files
- **Auto-discovery** - Automatically categorizes your files
- **Simple processing** - Just run the cells in order

---

## 📋 Step 1: Basic Configuration

Set your event details and S3 paths:

In [ ]:
# ========================================
# BASIC CONFIGURATION
# ========================================

# Event Details
EVENT_NAME = '202408_TropicalStorm_Debby'  # Your disaster event name
PRODUCT_NAME = 'landsat8'                   # Product type (sentinel1, sentinel2, landsat, etc.)

# S3 Paths
BUCKET = 'nasa-disasters'                                          # S3 bucket
SOURCE_PATH = f'drcs_activations/{EVENT_NAME}/{PRODUCT_NAME}'      # Where your files are
DESTINATION_BASE = 'drcs_activations_new'                          # Where to save COGs

# Processing Options
OVERWRITE = False  # Set to True to replace existing files
VERIFY = True      # Set to True to verify results after processing

print("✅ Basic configuration loaded")
print(f"Event: {EVENT_NAME}")
print(f"Source: s3://{BUCKET}/{SOURCE_PATH}")
print(f"Destination: s3://{BUCKET}/{DESTINATION_BASE}/")

## 🔍 Step 2: Connect to S3 and List Files

Let's see what files are available before configuring filename transformations:

In [ ]:
# Import necessary modules
import sys
import os
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path('..').resolve()))

# Import S3 operations
from core.s3_operations import (
    initialize_s3_client,
    list_s3_files,
    get_file_size_from_s3
)

# Initialize S3 client
print("🌐 Connecting to S3...")
s3_client, _ = initialize_s3_client(bucket_name=BUCKET, verbose=False)

if s3_client:
    print("✅ Connected to S3\n")
    
    # List all TIF files
    print(f"📂 Files in s3://{BUCKET}/{SOURCE_PATH}:")
    print("="*60)
    
    files = list_s3_files(s3_client, BUCKET, SOURCE_PATH, suffix='.tif')
    
    if files:
        print(f"Found {len(files)} .tif files:\n")
        for i, file_path in enumerate(files[:10], 1):  # Show first 10 files
            filename = os.path.basename(file_path)
            try:
                size_gb = get_file_size_from_s3(s3_client, BUCKET, file_path)
                print(f"{i:2}. {filename:<60} ({size_gb:.2f} GB)")
            except:
                print(f"{i:2}. {filename}")
        
        if len(files) > 10:
            print(f"\n... and {len(files) - 10} more files")
        
        print("\n" + "="*60)
        print("📝 Now you can see the file naming patterns!")
        print("   Use this information to create filename functions in the next step.")
    else:
        print("⚠️ No .tif files found in the specified path.")
        print("   Check your SOURCE_PATH configuration.")
else:
    print("❌ Could not connect to S3. Check your AWS credentials.")
    files = []

## 🏷️ Step 3: Define Filename Transformations

Based on the files you see above, define how to transform the filenames:

In [ ]:
# ========================================
# FILENAME GENERATION FUNCTIONS
# Modify these based on the files you see above
# ========================================

import re

def extract_date_from_filename(filename):
    """Extract date from filename in YYYYMMDD format."""
    dates = re.findall(r'\d{8}', filename)
    if dates:
        date_str = dates[0]
        return f"{date_str[0:4]}-{date_str[4:6]}-{date_str[6:8]}"
    return None

def create_truecolor_filename(original_path, event_name):
    """Create filename for trueColor products."""
    filename = os.path.basename(original_path)
    stem = os.path.splitext(filename)[0]
    date = extract_date_from_filename(stem)
    
    if date:
        stem_clean = re.sub(r'_\d{8}', '', stem)
        return f"{event_name}_{stem_clean}_{date}_day.tif"
    return f"{event_name}_{stem}_day.tif"

def create_colorinfrared_filename(original_path, event_name):
    """Create filename for colorInfrared products."""
    filename = os.path.basename(original_path)
    stem = os.path.splitext(filename)[0]
    date = extract_date_from_filename(stem)
    
    if date:
        stem_clean = re.sub(r'_\d{8}', '', stem)
        return f"{event_name}_{stem_clean}_{date}_day.tif"
    return f"{event_name}_{stem}_day.tif"

def create_naturalcolor_filename(original_path, event_name):
    """Create filename for naturalColor products."""
    filename = os.path.basename(original_path)
    stem = os.path.splitext(filename)[0]
    date = extract_date_from_filename(stem)
    
    if date:
        stem_clean = re.sub(r'_\d{8}', '', stem)
        return f"{event_name}_{stem_clean}_{date}_day.tif"
    return f"{event_name}_{stem}_day.tif"

def create_generic_filename(original_path, event_name):
    """Generic filename creator for any file type."""
    filename = os.path.basename(original_path)
    stem = os.path.splitext(filename)[0]
    date = extract_date_from_filename(stem)
    
    if date:
        stem_clean = re.sub(r'_\d{8}', '', stem)
        return f"{event_name}_{stem_clean}_{date}_day.tif"
    return f"{event_name}_{stem}_day.tif"

# Map product types to filename creators
FILENAME_CREATORS = {
    'trueColor': create_truecolor_filename,
    'colorInfrared': create_colorinfrared_filename,
    'naturalColor': create_naturalcolor_filename,
    # Add more mappings as needed based on your files
}

# Optional: Override output directories for each category
OUTPUT_DIRS = {
    'trueColor': 'Landsat/trueColor',
    'colorInfrared': 'Landsat/colorIR',
    'naturalColor': 'Landsat/naturalColor',
    # Add more as needed
}

# Optional: Manual no-data values (None = auto-detect)
NODATA_VALUES = {
    'NDVI': -9999,
    'MNDWI': -9999,
    # Leave empty or set to None for auto-detection
}

print("✅ Filename functions defined")

# Test with a sample filename if files exist
if files:
    sample_file = files[0]
    sample_name = os.path.basename(sample_file)
    
    # Try to detect which function to use
    if 'trueColor' in sample_name or 'truecolor' in sample_name:
        new_name = create_truecolor_filename(sample_file, EVENT_NAME)
    elif 'colorInfrared' in sample_name or 'colorIR' in sample_name:
        new_name = create_colorinfrared_filename(sample_file, EVENT_NAME)
    elif 'naturalColor' in sample_name:
        new_name = create_naturalcolor_filename(sample_file, EVENT_NAME)
    else:
        new_name = create_generic_filename(sample_file, EVENT_NAME)
    
    print(f"\n📝 Example transformation:")
    print(f"   Original: {sample_name}")
    print(f"   → New:    {new_name}")

## 🚀 Step 4: Initialize Processor and Preview

Now let's set up the processor and preview all transformations:

In [ ]:
# Import our simplified helper
from notebooks.notebook_helpers import SimpleProcessor

# Create full configuration
config = {
    'event_name': EVENT_NAME,
    'bucket': BUCKET,
    'source_path': SOURCE_PATH,
    'destination_base': DESTINATION_BASE,
    'overwrite': OVERWRITE,
    'verify': VERIFY,
    'filename_creators': FILENAME_CREATORS,
    'output_dirs': OUTPUT_DIRS,
    'nodata_values': NODATA_VALUES
}

# Initialize processor
processor = SimpleProcessor(config)

# Connect to S3 (already connected, but needed for processor)
if processor.connect_to_s3():
    print("✅ Processor ready\n")
    
    # Discover and categorize files
    num_files = processor.discover_files()
    
    if num_files > 0:
        # Show preview of transformations
        processor.preview_processing()
        
        print("\n📌 Review the transformations above.")
        print("   If they look incorrect, go back to Step 3 and adjust the filename functions.")
        print("   When ready, proceed to Step 5 to process the files.")
    else:
        print("⚠️ No files found to process.")
else:
    print("❌ Could not initialize processor.")

## ⚙️ Step 5: Process Files

Run this cell to start processing all files:

In [ ]:
# Process all files
if 'num_files' in locals() and num_files > 0:
    print("🚀 Starting processing...")
    print("This may take several minutes depending on file sizes.\n")
    
    # Process everything
    results = processor.process_all()
    
    # Display results
    if not results.empty:
        print("\n📊 Processing Complete!")
        display(results) if 'display' in dir() else print(results)
else:
    print("⚠️ No files to process. Complete Steps 1-4 first.")

## 📈 Step 6: Review Results (Optional)

View detailed results and statistics:

In [ ]:
# Analyze results
if 'results' in locals() and not results.empty:
    print("📊 PROCESSING STATISTICS")
    print("="*40)
    
    # Success rate
    total = len(results)
    success = len(results[results['status'] == 'success'])
    failed = len(results[results['status'] == 'failed'])
    skipped = len(results[results['status'] == 'skipped'])
    
    print(f"Total files: {total}")
    print(f"✅ Success: {success}")
    print(f"❌ Failed: {failed}")
    print(f"⏭️ Skipped: {skipped}")
    print(f"\nSuccess rate: {(success/total*100):.1f}%")
    
    # Failed files
    if failed > 0:
        print("\n❌ Failed files:")
        failed_df = results[results['status'] == 'failed']
        for idx, row in failed_df.iterrows():
            print(f"  - {row['file']}: {row.get('error', 'Unknown error')}")
    
    # Processing times
    if 'time_seconds' in results.columns:
        success_df = results[results['status'] == 'success']
        if not success_df.empty:
            avg_time = success_df['time_seconds'].mean()
            max_time = success_df['time_seconds'].max()
            print(f"\n⏱️ Timing:")
            print(f"Average: {avg_time:.1f} seconds per file")
            print(f"Slowest: {max_time:.1f} seconds")
else:
    print("No results to analyze. Run Step 5 first.")

## 💡 Tips & Troubleshooting

### Workflow Summary:
1. **Configure** basic settings (Step 1)
2. **List files** from S3 to see naming patterns (Step 2)
3. **Define functions** to transform filenames (Step 3)
4. **Preview** transformations (Step 4)
5. **Process** all files (Step 5)
6. **Review** results (Step 6)

### Common Issues:

1. **"No files found"**
   - Check `SOURCE_PATH` in Step 1
   - Verify bucket permissions
   - Ensure files have `.tif` extension

2. **Wrong filename transformations**
   - Review actual filenames in Step 2
   - Adjust functions in Step 3
   - Re-run Step 4 to preview

3. **Files being skipped**
   - Files already exist in destination
   - Set `OVERWRITE = True` in Step 1

4. **Processing errors**
   - Check AWS credentials
   - Verify S3 write permissions
   - Check available disk space for temp files

### Need More Control?

Use the full template at `disaster_processing_template.ipynb` for:
- Manual chunk configuration
- Custom compression settings
- Detailed memory management
- Advanced processing options